## Part 1: Loading data, summary statistics, and plotting/correlations

In [1]:
!python --version

Python 3.11.5


In [2]:
# 1.
from datetime import datetime as dt, timezone
import pandas as pd
from numpy import argmax
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.stattools import grangercausalitytests
import json


charging_station_data = pd.read_csv('data/charging_station_data.tsv', sep='\t', encoding='ISO-8859-1')

charging_cols = [
	# unique identifiers
	'id',
	'station_name',

	# associate using date
	'open_date',

	# associate using location
	'country',
	'state',
	# 'city',

	# filter using status
	'status_code',
]
print(charging_station_data.head(10))

policy_data = pd.read_csv('data/transportation_policy_data.tsv', sep='\t', encoding='ISO-8859-1')
print(policy_data.head(10))

policy_cols = [
	# unique identifiers
	'id',
	'title',

	# associate using date
	'significant_update_date',

	# associate using location
	'state',

	# categorization
	'type',
	'categories'
]

parsed_date_col = 'parsed_date'

                        station_name   open_date    id date_last_confirmed  \
0           LADWP - Truesdale Center  1999-10-15  1517          2023-09-14   
1    LADWP - West LA District Office  2020-02-28  1519          2023-01-10   
2      Los Angeles Convention Center  1995-08-30  1523          2023-01-10   
3      LADWP - John Ferraro Building  1999-10-15  1525          2023-09-14   
4         LADWP - Haynes Power Plant  2018-05-01  1531          2023-01-10   
5  LADWP - Harbor Generating Station  1999-10-15  1552          2023-01-10   
6                LADWP - Sylmar West  2016-01-01  1556          2023-01-10   
7          LADWP - EV Service Center  1999-10-15  1572          2023-01-10   
8             LADWP - Fairfax Center  2019-04-01  1573          2023-01-10   
9     California Air Resources Board  1996-10-15  1583          2022-09-14   

  expected_date status_code            updated_at   facility_type  \
0           NaN           E  2023-09-14T14:01:49Z         UTILITY   
1  

In [3]:
charging_station_data['open_date'].value_counts()['US']

12

In [4]:
charging_station_data.head(10)

,station_name,open_date,id,date_last_confirmed,expected_date,status_code,updated_at,facility_type,city,state,zip,street_address,country,ev_network
0,LADWP - Truesdale Center,1999-10-15,1517,2023-09-14,NaN,E,2023-09-14T14:01:49Z,UTILITY,Sun Valley,CA,91352,11797 Truesdale St,US,SHELL_RECHARGE
1,LADWP - West LA District Office,2020-02-28,1519,2023-01-10,NaN,E,2023-02-15T22:45:41Z,UTILITY,Los Angeles,CA,90024,1394 S Sepulveda Blvd,US,Non-Networked
2,Los Angeles Convention Center,1995-08-30,1523,2023-01-10,NaN,E,2023-02-14T15:54:11Z,PARKING_GARAGE,Los Angeles,CA,90015,1201 S Figueroa St,US,Non-Networked
3,LADWP - John Ferraro Building,1999-10-15,1525,2023-09-14,NaN,E,2023-09-14T14:01:49Z,UTILITY,Los Angeles,CA,90012,111 N Hope St,US,Non-Networked
4,LADWP - Haynes Power Plant,2018-05-01,1531,2023-01-10,NaN,E,2023-02-15T22:45:41Z,UTILITY,Long Beach,CA,90803,6801 E 2nd St,US,Non-Networked
5,LADWP - Harbor Generating Station,1999-10-15,1552,2023-01-10,NaN,E,2023-02-15T22:45:41Z,UTILITY,Wilmington,CA,90744,161 N Island Ave,US,Non-Networked
6,LADWP - Sylmar West,2016-01-01,1556,2023-01-10,NaN,E,2023-02-15T22:45:41Z,UTILITY,Sylmar,CA,91342,13201 Sepulveda Blvd,US,Non-Networked
7,LADWP - EV Service Center,1999-10-15,1572,2023-01-10,NaN,E,2023-02-15T22:45:41Z,UTILITY,Los Angeles,CA,90012,1630 N Main St,US,Non-Networked
8,LADWP - Fairfax Center,2019-04-01,1573,2023-01-10,NaN,E,2023-02-15T22:45:41Z,UTILITY,Los Angeles,CA,90016,2311 S Fairfax Ave,US,Non-Networked
9,California Air Resources Board,1996-10-15,1583,2022-09-14,NaN,E,2023-02-14T15:54:11Z,STATE_GOV,El Monte,CA,91731,9530 Telstar Ave,US,Non-Networked


In [5]:
def null_filtered(data, cols):
    mask = 1
    for col in cols:
        mask &= data[col].notnull()
        mask &= (data[col].ne('None'))
    return data[mask].reset_index()

def filter_invalid(data, col_name):
  mask = 1
  mask &= (data[col_name].ne('US'))
  mask &= (data[col_name].ne('CA'))
  return data[mask].reset_index()

def get_formatted_date(data_slice, date_col, format: str):
    return [
        dt.strptime(date_str, format)
        for date_str in data_slice[date_col]
    ]

In [6]:
charging_station_data = null_filtered(charging_station_data, charging_cols)[charging_cols]
charging_station_data = filter_invalid(charging_station_data, 'open_date')
#charging_station_data = charging_station_data.mask(charging_station_data.astype(object).eq('None')).dropna()

In [7]:
charging_station_data.head(10)

,index,id,station_name,open_date,country,state,status_code
0,0,1517,LADWP - Truesdale Center,1999-10-15,US,CA,E
1,1,1519,LADWP - West LA District Office,2020-02-28,US,CA,E
2,2,1523,Los Angeles Convention Center,1995-08-30,US,CA,E
3,3,1525,LADWP - John Ferraro Building,1999-10-15,US,CA,E
4,4,1531,LADWP - Haynes Power Plant,2018-05-01,US,CA,E
5,5,1552,LADWP - Harbor Generating Station,1999-10-15,US,CA,E
6,6,1556,LADWP - Sylmar West,2016-01-01,US,CA,E
7,7,1572,LADWP - EV Service Center,1999-10-15,US,CA,E
8,8,1573,LADWP - Fairfax Center,2019-04-01,US,CA,E
9,9,1583,California Air Resources Board,1996-10-15,US,CA,E


In [8]:
charging_station_data[parsed_date_col] = get_formatted_date(
    charging_station_data,
    'open_date',
    '%Y-%m-%d'
)

In [9]:
# run summary statistics on charging station data
charging_station_data.info()
charging_station_data.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78383 entries, 0 to 78382
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   index         78383 non-null  int64         
 1   id            78383 non-null  object        
 2   station_name  78383 non-null  object        
 3   open_date     78383 non-null  object        
 4   country       78383 non-null  object        
 5   state         78383 non-null  object        
 6   status_code   78383 non-null  object        
 7   parsed_date   78383 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 4.8+ MB


,index,id,station_name,open_date,country,state,status_code,parsed_date
0,0,1517,LADWP - Truesdale Center,1999-10-15,US,CA,E,1999-10-15


In [10]:
policy_data = null_filtered(policy_data, policy_cols)[policy_cols]
policy_data = filter_invalid(policy_data, 'significant_update_date')

# policy_data = policy_data.mask(policy_data.astype(object).eq('None')).dropna()

In [11]:
policy_data[parsed_date_col] = get_formatted_date(
    policy_data,
    'significant_update_date',
    '%m/%d/%Y'
)

In [12]:
policy_data.info()
policy_data.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   index                    1127 non-null   int64         
 1   id                       1127 non-null   int64         
 2   title                    1127 non-null   object        
 3   significant_update_date  1127 non-null   object        
 4   state                    1127 non-null   object        
 5   type                     1127 non-null   object        
 6   categories               1127 non-null   object        
 7   parsed_date              1127 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 70.6+ KB


,index,id,title,significant_update_date,state,type,categories,parsed_date
0,0,284,Congestion Mitigation and Air Quality (CMAQ) I...,11/29/2021,US,Incentives,"[{""code"": ""STATION"", ""title"": ""Alternative Fue...",2021-11-29


In [13]:
# helpers for plotting
def get_name(unformatted):
	return unformatted

def make_scatterplot(slice, x_col, y_col, title):
	sns.scatterplot(data=slice, x=x_col, y=y_col)
	plt.title(title)
	plt.xlabel(get_name(x_col))
	plt.ylabel(get_name(y_col))
	plt.show()

def make_boxplot(slice, x_col, y_col, title):
	sns.boxplot(data=slice, x=x_col, y=y_col)
	plt.title(title)
	plt.xlabel(get_name(x_col))
	plt.ylabel(get_name(y_col))
	plt.show()

def make_vertical_boxplot(slice, col, title):
	_, ax = plt.subplots(figsize=(6, 3), sharey=True, sharex=True)
	ax.spines['top'].set_visible(False)
	ax.spines['right'].set_visible(False)
	ax.spines['left'].set_visible(False)
	ax.set_title(title)
	ax.boxplot(slice[col])

def make_barplot(slice, x_col, y_col, x_label, y_label, title):
	sns.barplot(data=slice, x=x_col, y=y_col, hue=x_col, legend=False)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.xticks(rotation=45)
	plt.show()

def make_lineplot(slice, x_col, y_col, x_label, y_label, title):
	sns.lineplot(data=slice, x=x_col, y=y_col, legend=False)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.xticks(rotation=45)
	plt.show()

def filter_by_state(data_slice, state):
	return data_slice[data_slice['state'] == state].reset_index()

def filter_outliers(data_slice, col, limit):
	return data_slice[data_slice[col] < limit]

def group_by_year(data_slice, date_col):
	grouped_slice = data_slice.groupby(data_slice[date_col].dt.year).count()
	grouped_slice = grouped_slice.drop(columns=date_col)
	return grouped_slice

In [14]:
# Initial year-based correlations by state

stations_per_year_by_state = {}
policies_per_year_by_state = {}


# overall
us_stations = charging_station_data[charging_station_data['country'] == 'US']
us_stations = group_by_year(charging_station_data, parsed_date_col)
# make_barplot(
# 	us_stations,
# 	parsed_date_col,
# 	'id',
# 	'Year',
# 	'Charging Stations built',
# 	'Countrywide Stations'
# )

federal_policies = filter_by_state(policy_data, 'US')
federal_policies = group_by_year(policy_data, parsed_date_col)
# make_barplot(
# 	federal_policies,
# 	parsed_date_col,
# 	'id',
# 	'Year',
# 	'Policies Enacted',
# 	'Federal Policies'
# )

station_states = set(charging_station_data['state'])
policy_states = set(policy_data['state'])

# 50 US states plus washington DC that we have policy and station data for
state_intersections = station_states & policy_states

# Canadian provinces for which we have station data
# station_provinces = station_states - state_intersections

# by state
for state in sorted(state_intersections):
	# charging stations built by year by state
	state_stations = filter_by_state(
		charging_station_data,
		state
	)
	state_stations = group_by_year(state_stations, parsed_date_col)
	# make_barplot(
	# 	state_stations,
	# 	parsed_date_col,
	# 	'id',
	# 	'Year',
	# 	'Charging Stations built',
	# 	f'{state} Stations'
	# )
	stations_per_year_by_state[state] = state_stations

	# # policies enacted per year by state
	state_policies = filter_by_state(
		policy_data,
		state
	)
	state_policies = group_by_year(state_policies, parsed_date_col)
	# make_barplot(
	# 	state_policies,
	# 	parsed_date_col,
	# 	'id',
	# 	'Year',
	# 	'Policies Enacted',
	# 	f'{state} Policies'
	# )
	policies_per_year_by_state[state] = state_policies



In [15]:
# investigate policies by category: move fields in json objects into their own columns

codes_to_names = {
	# technology types
	'BIOD': 'Biodiesel',
	'ETH': 'Ethanol',
	'NG': 'Natural Gas',
	'LPG': 'Liquified Petroleum Gas',
	'HY': 'Hydrogen',
	'ELEC': 'Electric Vehicles',
	'PHEV': 'Plug-In Electric Hybrid Vehicles',
	'HEV': 'Hybrid Electric Vehicles',
	'NEVS': 'Neighborhood Electric Vehicles',
	'RD': 'Renewable Diesel',
	'AFTMKTCONV': 'Aftermarket Conversions',
	'EFFEC': 'Fuel Economy / Efficiency',
	'IR': 'Idle Reduction',
	'AUTONOMOUS': 'Autonomous Vehicles',

	# incentive types
	'GNT': 'Grants',
	'TAX': 'Tax Incentives',
	'LOANS': 'Loans and Leases',
	'RBATE': 'Rebates',
	'EXEM': 'Exemptions',
	'TOU': 'Time-of-Use Rate',

	# regulation types
	'REQ': 'Acquisition / Fuel Use',
	'DREST': 'Driving / Idling',
	'REGIS': 'Registration / Licensing',
	'EVFEE': 'EV Registration Fee',
	'FUEL': 'Fuel Taxes',
	'STD': 'Fuel Production / Quality',
	'RFS': 'Renewable Fuel Standard / Mandate',
	'AIRQEMISSIONS': 'Air Quality / Emissions',
	'CCEINIT': 'Climate Change / Energy Initiatives',
	'UTILITY': 'Utility Definition',
	'BUILD': 'Building Codes',
	'RTC': 'Right-to-Charge',

	# user types
	'FLEET': 'Commercial',
	'GOV': 'Government Entity',
	'TRIBAL':  'Tribal Government',
	'IND': 'Personal Vehicle Owner or Driver',
	'STATION': 'Alternative Fuel Infrastructure Operator',
	'AFP': 'Alternative Fuel Producer',
	'PURCH': 'Alternative Fuel Purchaser',
	'MAN': 'Manufacturer',
	'MUD': 'Multi-Unit Dwelling',
	'TRANS': 'Transit',

	'OTHER': 'Other',
}

def get_code_name(code):
	return codes_to_names[code]

policy_data['parsed_categories'] = policy_data['categories'].apply(json.loads)
category_types = set()
for categories in policy_data['parsed_categories']:
	for category in categories:
		category_types.add(category['category_type'])

category_cols = []
for category_type in category_types:
	def get_category_codes_by_type(entry):
		codes = set()
		for category in entry:
			if category['category_type'] == category_type:
				codes.add(category['code'])
		return codes if codes else None

	category_col = f'{category_type}_types'
	category_cols.append(category_col)
	policy_data[category_col] = policy_data['parsed_categories'].apply(get_category_codes_by_type)

types_slice = policy_data[list(map(lambda x: f'{x}_types', category_types))]
types_slice.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   regulation_types  486 non-null    object
 1   tech_types        1127 non-null   object
 2   user_types        1118 non-null   object
 3   incentive_types   636 non-null    object
dtypes: object(4)
memory usage: 35.3+ KB


In [16]:

# gather exhaustive set of code values observed for each field
unique_codes = {}
for col in category_cols:
	print(col)
	values = set()
	for code_set in policy_data[col]:
		values |= {get_code_name(code) for code in code_set} if code_set else set()
	print('{\n\t' + '\n\t'.join(values) + '\n}\n')
	unique_codes[col] = values


regulation_types
{
	Acquisition / Fuel Use
	Other
	Building Codes
	Climate Change / Energy Initiatives
	Fuel Production / Quality
	Driving / Idling
	Right-to-Charge
	Registration / Licensing
	Utility Definition
	Air Quality / Emissions
	Fuel Taxes
	EV Registration Fee
}

tech_types
{
	Natural Gas
	Hydrogen
	Renewable Diesel
	Fuel Economy / Efficiency
	Biodiesel
	Liquified Petroleum Gas
	Idle Reduction
	Aftermarket Conversions
	Other
	Autonomous Vehicles
	Neighborhood Electric Vehicles
	Plug-In Electric Hybrid Vehicles
	Ethanol
	Electric Vehicles
	Hybrid Electric Vehicles
}

user_types
{
	Other
	Tribal Government
	Manufacturer
	Alternative Fuel Purchaser
	Government Entity
	Alternative Fuel Infrastructure Operator
	Personal Vehicle Owner or Driver
	Multi-Unit Dwelling
	Alternative Fuel Producer
	Commercial
	Transit
}

incentive_types
{
	Other
	Exemptions
	Time-of-Use Rate
	Loans and Leases
	Grants
	Tax Incentives
	Rebates
}



In [17]:
# Simple initial test
def time_to_ts(date_obj):
	return date_obj.replace(tzinfo=timezone.utc).timestamp()

def diy_index(df):
	return list(range(1, df.shape[0] + 1))

def filter_entries_outside_date_range(df, start, end):
	mask = 1
	mask &= df[parsed_date_col] >= start
	mask &= df[parsed_date_col] <= end
	return df[mask]

charging_station_data['open_timestamp'] = charging_station_data[parsed_date_col].apply(time_to_ts)
# charging_station_data.head()
state = 'NY'
stations = filter_by_state(charging_station_data, state)[[parsed_date_col]].sort_values(by=parsed_date_col).reset_index()
stations['station_counts'] = diy_index(stations)

# make_lineplot(
# 	stations,
# 	parsed_date_col,
# 	'station_counts',
# 	'Years',
# 	'Total Station Count',
# 	'NY Station Growth'
# )
policies = filter_by_state(policy_data, state)[[parsed_date_col]].sort_values(by=parsed_date_col).reset_index()
print(policies.head())
policies['policy_counts'] = diy_index(policies)
# make_lineplot(
# 	policies,
# 	parsed_date_col,
# 	'policy_counts',
# 	'Years',
# 	'Total Policy Count',
# 	'NY Policy Growth'
# )

highest_start_date = max(stations[parsed_date_col][0], policies[parsed_date_col][0])
# print(highest_start_date)
lowest_end_date = min(stations[parsed_date_col][stations.shape[0]-1], policies[parsed_date_col][policies.shape[0]-1])
# print(lowest_end_date)

# stations = filter_entries_outside_date_range(stations, highest_start_date, lowest_end_date)
# policies = filter_entries_outside_date_range(policies, highest_start_date, lowest_end_date)


# make_lineplot(
# 	stations,
# 	parsed_date_col,
# 	'station_counts',
# 	'Years',
# 	'Total Station Count',
# 	'Filtered NY Station Growth'
# )

# make_lineplot(
# 	policies,
# 	parsed_date_col,
# 	'policy_counts',
# 	'Years',
# 	'Total Policy Count',
# 	'Filtered NY Policy Growth'
# )

# merged_data['pct_diff_station_counts'] = merged_data['station_counts'].pct_change()
# merged_data['pct_diff_policy_counts'] = merged_data['policy_counts'].pct_change()
# merged_data = merged_data.dropna()
# merged_data.head()

#merged_data = merged_data[['station_counts', 'policy_counts']].pct_change().dropna()

# make_lineplot(
# 	merged_data,
# 	parsed_date_col,
# 	'pct_diff_station_counts',
# 	'Years',
# 	'% Change in Station Count',
# 	'NY Station Growth Rate'
# )

# make_lineplot(
# 	merged_data,
# 	parsed_date_col,
# 	'pct_diff_policy_counts',
# 	'Years',
# 	'% Change in Policy Count',
# 	'NY Policy Growth Rate'
# )

   index parsed_date
0      2  2015-06-12
1      3  2017-04-03
2      1  2018-06-25
3      5  2018-07-09
4      6  2018-08-08


In [18]:
# merged_data = pd.merge_asof(stations, policies, on=parsed_date_col)

# merged_data['station_counts_pct_change'] = merged_data['station_counts'].pct_change()

# merged_data['policy_counts_pct_change'] = merged_data['policy_counts'].pct_change()
# # merged_data['policy_counts_diff'].value_counts()
# merged_data_pct = merged_data[['station_counts_pct_change', 'policy_counts_pct_change']].dropna()

In [19]:
# results = grangercausalitytests(
# 	merged_data_pct,
# 	4
# )

In [20]:
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=1
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table
    are the P-Values. P-Values lesser than the significance level (0.05), implies
    the Null Hypothesis that the coefficients of the corresponding past values is
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(merged_data_pct, variables = merged_data_pct.columns)

NameError: name 'merged_data_pct' is not defined

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05):
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(merged_data_pct)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
station_counts_pct_change ::  536.29    > 12.3212   =>   True
policy_counts_pct_change ::  0.0       > 4.1296    =>   False


In [ ]:
nobs = int(0.2*len(merged_data_pct))
df_train, df_test = merged_data_pct[0:-nobs], merged_data_pct[-nobs:]

# Check size
print(df_train.shape)
print(df_test.shape)

(3096, 2)
(773, 2)


In [ ]:
# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

# check that time series data is stationary (mean and variance do not change over time)
# to do this, we are using Augmented Dicker-Fuller Test (ADF Test)
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue']
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [ ]:
# ADF Test on each column
for name, column in df_train.items():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "station_counts_pct_change" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -0.9848
 No. Lags Chosen       = 15
 Critical value 1%     = -3.432
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.7588. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "policy_counts_pct_change" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -21.1625
 No. Lags Chosen       = 5
 Critical value 1%     = -3.432
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.




In [ ]:
# we must pick the order that gives a model with least AIC.

model = VAR(df_train)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  -34.21943964779387
BIC :  -34.207735202200745
FPE :  1.3762146848821863e-15
HQIC:  -34.21523626118642 

Lag Order = 2
AIC :  -45.9644351612454
BIC :  -45.94492249146152
FPE :  1.0911876653147119e-20
HQIC:  -45.957427512530934 

Lag Order = 3
AIC :  -56.93709611115495
BIC :  -56.909771004521396
FPE :  1.8729809061549678e-25
HQIC:  -56.92728259515629 

Lag Order = 4
AIC :  -67.35490262271314
BIC :  -67.3197608628895
FPE :  5.5993482105257514e-30
HQIC:  -67.34228163280926 

Lag Order = 5
AIC :  -76.04504952004231
BIC :  -76.00208688700222
FPE :  9.420081137257265e-34
HQIC:  -76.02961944816654 

Lag Order = 6
AIC :  -75.92919229168268
BIC :  -75.87840456170947
FPE :  1.0577202414591156e-33
HQIC:  -75.91095152832096 

Lag Order = 7
AIC :  -75.09536629619971
BIC :  -75.03674924188195
FPE :  2.4349882359180945e-33
HQIC:  -75.07431323038884 

Lag Order = 8
AIC :  -74.19050680149137
BIC :  -74.1240561917184
FPE :  6.018279913132997e-33
HQIC:  -74.1666398208172 

Lag Order =

/Users/brennanmcmanus/fall23/empirical/empirical-env/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
# we will take Lago Order = 5 since it has lowest AIC
model_fitted = model.fit(7)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 30, Nov, 2023
Time:                     11:37:32
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -75.0367
Nobs:                     3089.00    HQIC:                  -75.0743
Log likelihood:           107249.    FPE:                2.43499e-33
AIC:                     -75.0954    Det(Omega_mle):     2.41151e-33
--------------------------------------------------------------------
Results for equation station_counts_pct_change
                                  coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------------
const                                0.000000         0.000000           85.390           0.000
L1.station_counts_pct_change         3.489757         0.000001      3050200.

In [23]:
# Add registration data
import json
import os

registration_data = {}
registration_states = []
for root, _, filenames in os.walk(f'data/granular_ev_registrations', topdown=False):
	for filename in filenames:
		state_str = filename.split('_')[0].upper()
		registration_states.append(state_str)

		filepath = f'{root}/{filename}'

		state_reg_data = pd.read_csv(filepath, sep=',', encoding='ISO-8859-1')
		registration_data[state_str] = state_reg_data

registration_states.sort()

/var/folders/rt/hw8f7zhs1hq2h1fft4tvgnh80000gn/T/ipykernel_49224/3178787428.py:14: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  state_reg_data = pd.read_csv(filepath, sep=',', encoding='ISO-8859-1')
/var/folders/rt/hw8f7zhs1hq2h1fft4tvgnh80000gn/T/ipykernel_49224/3178787428.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  state_reg_data = pd.read_csv(filepath, sep=',', encoding='ISO-8859-1')
/var/folders/rt/hw8f7zhs1hq2h1fft4tvgnh80000gn/T/ipykernel_49224/3178787428.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  state_reg_data = pd.read_csv(filepath, sep=',', encoding='ISO-8859-1')
/var/folders/rt/hw8f7zhs1hq2h1fft4tvgnh80000gn/T/ipykernel_49224/3178787428.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  state_reg_data = pd.read_csv(filepath, sep=',

In [24]:
# def null_filtered(data, cols):
#     mask = 1
#     for col in cols:
#         mask &= data[col].notnull()
#         mask &= (data[col].ne('None'))
#     return data[mask].reset_index()

# def get_formatted_date(data_slice, date_col, format: str):
#     return [
#         dt.strptime(date_str, format)
#         for date_str in data_slice[date_col]
#     ]

date_formats = {
	'CA': '%Y-%m-%d',
	'CO': '%m/%d/%Y',
	'CT': '%m/%d/%Y',
	'FL': '%m/%d/%Y',
	'ME': '%m/%d/%Y',
	'MN': '%m/%d/%Y',
	'MT': '%m/%d/%Y',
	'NC': '%m/%d/%Y',
      	'NJ': '%m/%d/%Y',
        'NY': '%m/%d/%Y',
        'OR': '%m/%d/%Y',
        'TN': '%m/%d/%Y',
        'TX': '%m/%d/%Y',
        'VA': '%m/%d/%Y',
	'VT': '%m/%d/%Y',
      	'WA': '%m/%d/%Y',
	'WI': '%m/%d/%Y',     
}

registration_cols = {
	'CA': {
		'parsed_date': 'Registration Valid Date'
	},
	'CO': {
		'parsed_date': 'Registration Date'
	},
	'CT': {
		'parsed_date': 'Registration Date'
	},
	'FL': {
		'parsed_date': 'Registration Valid Date'
	},
	'ME': {
		'parsed_date': 'Registration Date'
	},
	'MN': {
		'parsed_date': 'Registration Date'
	},
	'MT': {
		'parsed_date': 'Registration Date'
	},
	'NC': {
		'parsed_date': 'Registration Date'
	},
	'NJ': {
		'parsed_date': 'Registration Date'
	},
      	'NY': {
		'parsed_date': 'Registration Date'
	},
	'OR': {
		'parsed_date': 'Registration Date'
	},
      	'TN': {
		'parsed_date': 'Registration Date'
	},
      	'TX': {
		'parsed_date': 'Registration Date'
	},
      	'VA': {
		'parsed_date': 'Registration Date'
	},
     	'VT': {
		'parsed_date': 'Registration Date'
	},
      	'WA': {
		'parsed_date': 'Registration Valid Date'
	},
	'WI': {
		'parsed_date': 'Registration Valid Date'
	}
}

for i, state in enumerate(registration_states):
	state_reg_data = registration_data[state]
	parsed_cols = registration_cols[state].keys()

	for parsed_col, raw_col in registration_cols[state].items():
		state_reg_data[parsed_col] = state_reg_data[raw_col]

	state_reg_data = null_filtered(state_reg_data, parsed_cols)
	state_reg_data['parsed_date'] = get_formatted_date(
		state_reg_data,
		'parsed_date',
		date_formats[state]
	)
	state_reg_data = state_reg_data[parsed_cols]
	state_reg_data['parsed_state'] = state
      
	registration_data[state] = state_reg_data


In [25]:
for state in registration_data.keys():
	print(registration_data[state].head(900))


    parsed_date parsed_state
0    2019-06-07           WA
1    2019-06-07           WA
2    2019-06-07           WA
3    2019-06-07           WA
4    2019-06-07           WA
..          ...          ...
895  2019-06-07           WA
896  2019-06-07           WA
897  2019-06-07           WA
898  2019-06-07           WA
899  2019-06-07           WA

[900 rows x 2 columns]
    parsed_date parsed_state
0    2017-12-01           NJ
1    2017-12-01           NJ
2    2017-12-01           NJ
3    2017-12-01           NJ
4    2017-12-01           NJ
..          ...          ...
895  2017-06-01           NJ
896  2017-06-01           NJ
897  2017-06-01           NJ
898  2017-06-01           NJ
899  2017-06-01           NJ

[900 rows x 2 columns]
    parsed_date parsed_state
0    2022-10-01           TX
1    2022-10-01           TX
2    2022-10-01           TX
3    2022-10-01           TX
4    2022-10-01           TX
..          ...          ...
895  2022-04-01           TX
896  2022-04-01         

In [26]:

registrations = registration_data['NY']

print(policies[parsed_date_col].head())
highest_start_date = max([
	stations[parsed_date_col][0],
	policies[parsed_date_col][0],
	registrations[parsed_date_col][0],
])
# print(highest_start_date)
lowest_end_date = min([
	stations[parsed_date_col][stations.shape[0]-1],
	policies[parsed_date_col][policies.shape[0]-1],
	registrations[parsed_date_col][registrations.shape[0]-1]
])
# print(lowest_end_date)

# stations = filter_entries_outside_date_range(stations, highest_start_date, lowest_end_date)
# policies = filter_entries_outside_date_range(policies, highest_start_date, lowest_end_date)
# registrations = filter_entries_outside_date_range(registrations, highest_start_date, lowest_end_date)

stations['new_stations'] = 1
policies['new_policies'] = 1
registrations['new_registrations'] = 1

merged_cols = ['new_stations', 'new_policies', 'new_registrations']

merged_data = pd.merge(stations, policies, how='outer', on=parsed_date_col)
merged_data = pd.merge(merged_data, registrations, how='outer', on=parsed_date_col)
merged_data = merged_data[['parsed_date'] + merged_cols].fillna(0)

merged_data['quarter_year'] = merged_data['parsed_date'].dt.year.map(str) + '_Q' + merged_data['parsed_date'].dt.quarter.map(str)
merged_data = merged_data[['quarter_year'] + merged_cols]
quarterly = merged_data.groupby(merged_data['quarter_year'], as_index=False).agg(sum)

quarterly.head(100)

# type(merged_data[parsed_date_col][0])

0   2015-06-12
1   2017-04-03
2   2018-06-25
3   2018-07-09
4   2018-08-08
Name: parsed_date, dtype: datetime64[ns]


/var/folders/rt/hw8f7zhs1hq2h1fft4tvgnh80000gn/T/ipykernel_49224/324555923.py:33: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  quarterly = merged_data.groupby(merged_data['quarter_year'], as_index=False).agg(sum)


,quarter_year,new_stations,new_policies,new_registrations
0,2008_Q3,2.0,0.0,0.0
1,2009_Q4,1.0,0.0,0.0
2,2010_Q1,2.0,0.0,0.0
3,2010_Q4,1.0,0.0,0.0
4,2011_Q1,0.0,0.0,725.0
5,2011_Q4,71.0,0.0,0.0
6,2012_Q1,5.0,0.0,1082.0
7,2012_Q2,2.0,0.0,0.0
8,2012_Q4,7.0,0.0,0.0
9,2013_Q1,2.0,0.0,2582.0


In [28]:
def get_lookback_for_col(data_slice, col, quarters):
	series = data_slice[col]
	lookbacks = []
	for i, case in enumerate(series):
		# sum up counts from col from cases within the past <quarters> quarters
		if i - quarters <= 0:
			lookbacks.append(None)
		else:
			lookbacks.append(
				sum([series[i - j] for j in range (1, quarters + 1)])
			)
	return lookbacks

DESIRED_LOOKBACK_MAX = 8
for i in range(1, DESIRED_LOOKBACK_MAX + 1):
	for col in merged_cols:
		col_name = 'prev_' + col.split('_')[1] + f'_{i}Q'
		quarterly[col_name] = get_lookback_for_col(quarterly, col, i)
quarterly.head(40)

quarterly.to_csv(path_or_buf='data/quarterly_merged_counts.csv', sep=',')
